# Compute projections
Best options is to run `iss project-round` command after each acquisition:

`iss project-round -p $DATAPATH -n $PREFIX`

where `$DATAPATH` is the path to the data relative to raw data root and `$PREFIX` is the file name prefix and directory name of the round, e.g. `genes_round_1_1`.


# Import requirements

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import iss_preprocess as iss
import numpy as np
import matplotlib.pyplot as plt
import yaml
from flexiznam.config import PARAMETERS
from pathlib import Path
from itertools import cycle
from matplotlib.animation import FuncAnimation

# Set options and path

In [ ]:
data_path = "becalia_rabies_barseq/BRAC8501.6a/chamber_06"

processed_path = Path(PARAMETERS["data_root"]["processed"])
# metadata = iss.io.load_metadata(data_path)

ops = iss.io.load.load_ops(data_path)

In [ ]:
prefix = "mCherry_1"

In [ ]:
# ops['camera_order'] = [1 ,3,4,2]
for cam, ch in enumerate(np.argsort(ops["camera_order"])):
    print(ch, cam)

In [ ]:
roi_dims = iss.io.load.get_roi_dimensions(data_path, prefix=prefix)
c0 = 1
c1 = round(roi_dims[0, 1] / 2)
c2 = round(roi_dims[0, 2] / 2)
ref_coords = [c0, c1, c2]

shift_right, shift_down, tile_shape = iss.pipeline.register_adjacent_tiles(
    data_path, ref_coors=[1, 3, 3], prefix="mCherry_1", ref_ch=1
)
print(shift_right, shift_down, tile_shape)

In [ ]:
iss.vis.vis.plot_overview_images(
    data_path=data_path,
    prefix=prefix,
    plot_grid=True,
    downsample_factor=5,
    save_raw=True,
    group_channels=False,
    use_slurm=False,
)

In [ ]:
prefix = "mCherry_1"
import cv2

fig = plt.figure(figsize=(20, 20))

for roi in np.arange(8):
    fig.clear()
    print(f"Doing roi {roi+1}")
    print("   ... stitching", flush=True)
    stack = iss.pipeline.stitch_tiles(
        data_path,
        prefix,
        roi=roi + 1,
        suffix="max",
        ich=1,
        correct_illumination=False,
    )
    print("   ... downsampling", flush=True)
    downsampled_size = (np.array(stack.shape) / 20).astype("int")
    downsampled = cv2.resize(
        stack, downsampled_size[::-1], interpolation=cv2.INTER_LINEAR
    )
    print("   ... plotting", flush=True)
    plt.imshow(stack[::10, ::10], vmax=150)
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.axis("off")
    print("   ... saving", flush=True)
    fig.savefig(
        processed_path / data_path / f"mcherry_overview_roi{roi + 1:02d}.png", dpi=600
    )

# Montage

A montage can be generated from the overviews using ImageMagick:

```
ml ImageMagick
cd MOUSE_DIRECTORY
magick montage -font "DejaVu-Sans" round_overviews/chamber_08_roi_*_genes_round_5*.png  -geometry 2000x -tile 5x2 -background black  chamber_08_genes_round_5_overview.png
```

The 2000x sets the number of pixels of each image in the montage (in x) and  4x2 the tile
arangment.

To merge all round

```
magick montage -font "DejaVu-Sans" chamber_08_genes_round*_overview.png  -geometry x800 -tile 1x -background black  chamber_08_all_genes_rounds.png
```

To add file name:

`magick montage -label "%f" -font "DejaVu-Sans" round_overviews/chamber_08_roi_*_genes_round_2*.png  -geometry 2000x -tile 5x2 chamber_08_genes_round_2_overview.png`

To flip rows:

`magick montage round_overviews/chamber_08_roi_17*.png round_overviews/chamber_08_roi_18*.png round_overviews/chamber_08_roi_19*.png round_overviews/chamber_08_roi_20*.png round_overviews/chamber_08_roi_21*.png  -geometry 2000x -tile 5x1 chamber_08_overview_top_row.png`

and

`magick montage round_overviews/chamber_08_roi_12*.png round_overviews/chamber_08_roi_13*.png round_overviews/chamber_08_roi_14*.png round_overviews/chamber_08_roi_15*.png round_overviews/chamber_08_roi_16*.png  -geometry 2000x -tile 5x1 chamber_08_overview_bottom_row.png`

`magick montage chamber_08_overview_top_row.png chamber_08_overview_bottom_row.png -geometry 10000x -tile 1x2 chamber_08_overview.png`

